In [1]:
import pickle 
import pandas as pd 
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly (use `wandb login --relogin` to force relogin)


True

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

67233


In [5]:
REBUILD_DATA = True
class GRV():
    #class to store training data
    
    #reading data from pickle
    file = open( r"DATA/total_list.npy", 'rb')
    data = pickle.load(file)
    #seperate out classes from inputs
    raceIDs,inputs,classes = zip(*data)
    
    #removing nan from inputs and convert to float
    inputs_df = pd.DataFrame(inputs)
    inputs_df.fillna(value=-1,inplace = True)
    inputs = inputs_df.values.tolist()
    inputs = [[float(i) for i in j] for j in inputs]
    
    
    
    #data
    training_data = []

    def make_training_data(self):
        excluded = 0
        for i in range(len(self.inputs)):
            if len(self.classes[i]) == 8:
                self.training_data.append([np.array(self.inputs[i]),np.array(self.classes[i])])
            else:
                adjustedList = self.classes[i]+([8]*(8-len(self.classes[i])))
                self.training_data.append([np.array(self.inputs[i]),np.array(adjustedList)])
                if len(adjustedList) != 8:
                    print(adjustedList)
        np.save('training_data.npy', self.training_data)
        print("excluded = ", excluded)

if REBUILD_DATA:
    grv = GRV()
    grv.make_training_data()

In [12]:
#dataset setup
training_data = grv.training_data

X = torch.Tensor([i[0] for i in training_data])
Y = torch.Tensor([i[1] for i in training_data])

#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
    
Y_w = torch.tensor([i for i in Y_w])

my_dataset = TensorDataset(X,Y_w) 
my_dataloader = DataLoader(my_dataset)

In [23]:
config = dict(
    epochs=50,
    classes=8,
    kernels=[16, 32],
    batch_size=128,
    learning_rate=0.0001,
    validation_split = 0.1,
    dataset="MNIST",
    architecture="CNN",
    dataset_size = len(training_data))

config["dataset_size"]

67233

In [24]:
#have to pass in dataset to get_data (created above)
def make_loader(dataset, config, train=True):
	dataset_size = len(dataset)
	indices = list(range(dataset_size))
	split = int(np.floor(config["validation_split"] * dataset_size))
	random_seed= 42
	np.random.seed(random_seed)
	np.random.shuffle(indices)
	train_indices, val_indices = indices[split:], indices[:split]

	if train:
		dataset_sampler = SubsetRandomSampler(train_indices)
		loader = torch.utils.data.DataLoader(dataset=dataset,
										batch_size=config["batch_size"], 
										shuffle=False,
										pin_memory=True, num_workers=2, sampler = dataset_sampler)
	else:
		dataset_sampler  = SubsetRandomSampler(val_indices)
		loader = torch.utils.data.DataLoader(dataset=dataset,
								shuffle=False,
								pin_memory=True, num_workers=2, sampler = dataset_sampler)


	
	return loader

							

In [27]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(81, 256)
        self.fc2 = nn.Linear(256, 32)
        self.fc3 = nn.Linear(32, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x #nn.Softmax(x, dim=1)
    

In [12]:
def make(config, dataset):
    # Make the data

    train_loader = make_loader(dataset,config, train=True)
    test_loader = make_loader(dataset,config, train=False)
    # Make the model
    model = Net().to(device)

    # Make the loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["learning_rate"])
    
    return model, train_loader, test_loader, criterion, optimizer

In [13]:
make(config,my_dataset)

(Net(
   (fc1): Linear(in_features=81, out_features=64, bias=True)
   (fc2): Linear(in_features=64, out_features=32, bias=True)
   (fc3): Linear(in_features=32, out_features=8, bias=True)
 ),
 MSELoss(),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.0001
     weight_decay: 0
 ))

In [15]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            #print(f"{images=},\n,{labels=}")
            _, predicted = torch.max(outputs.data, 1)
            _,real = torch.max(labels.data,1)
            #print(f"{real.item(),predicted.item()=}")
            total += labels.size(0)
            correct += (predicted.item() == real.item())
            #print(f"{correct=}")

        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")
        
        wandb.log({"test_accuracy": correct / total})

    # Save the model in the exchangeable ONNX format
    #torch.onnx.export(model, images, "model.onnx")
    #wandb.save("model.onnx")


In [26]:
def train(model, loader,test_loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (images, labels) in enumerate(loader):

            loss = train_batch(images, labels, model, optimizer, criterion)
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 250) == 0:
                train_log(loss, example_ct, epoch)

        if epoch %5 ==0:
            test(model,test_loader)

def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)
    
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels.float())
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [17]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [18]:
def model_pipeline(hyperparameters, dataset):

    # tell wandb to get started
    with wandb.init(project="pytorch-demo", config=hyperparameters):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config, dataset)
      print(model)

      # and use them to train the model
      train(model, train_loader,test_loader, criterion, optimizer, config)

      # and test its final performance
      test(model, test_loader)

    return model

In [28]:
config = dict(
    epochs=50,
    classes=8,
    kernels=[81,256,32,8],
    batch_size=128,
    learning_rate=0.0001,
    validation_split = 0.1,
    dataset="MNIST",
    architecture="CNN",
    dataset_size = len(training_data))

config["dataset_size"]

model = model_pipeline(config, my_dataset)

  0%|          | 0/50 [00:00<?, ?it/s]

Net(
  (fc1): Linear(in_features=81, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=8, bias=True)
)
Loss after 31872 examples: 0.158


  2%|▏         | 1/50 [00:10<08:30, 10.41s/it]

Accuracy of the model on the 6723 test images: 13.550498289454113%
Loss after 63838 examples: 0.121
Loss after 95838 examples: 0.116


  4%|▍         | 2/50 [00:12<04:25,  5.54s/it]

...

 98%|█████████▊| 49/50 [03:12<00:03,  3.20s/it]

Loss after 2974206 examples: 0.104
Loss after 3006206 examples: 0.108


100%|██████████| 50/50 [03:14<00:00,  3.89s/it]


Accuracy of the model on the 6723 test images: 23.888145173285736%



epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▃▂▂▂▂▁▁▁▂▁▁▂▁▁▁▁▂▁▂▂▂▂▁▂▁▁▁▁▂▂▂▁▁▁▁▁▁▁▂
test_accuracy,▁▆▅▆▆▅▇▇▇██
epoch,49
loss,0.10838
test_accuracy,0.23888


In [ ]:
#Generate winner only class
Y_w = []
for i in Y:
    n = np.zeros(8)
    index = torch.argmin(i)
    n[index] = float(1)
    Y_w.append(n)
Y_w = torch.tensor([i for i in Y_w])
Y_w[1:10]

In [ ]:
#indexed win only
Y_wi = []
for i in Y:
    #print(torch.argmin(i))
    Y_wi.append(torch.argmin(i))
Y_w = torch.tensor(Y_wi)
print(Y_w)

In [ ]:
#Generates places
Y_p = []
for i in Y:
    place = [x*x for x in i]
    Y_p.append(place)
    
Y_p[1]

In [ ]:
#validating model is new
for (i,j) in net.named_parameters():
    print(i,j)

In [ ]:
winners = [i.item() for i in Y_wi]
from collections import Counter
  
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
    
List = winners
print(most_frequent(List))